# Shared Memory Parallelism using OpenMP

We want to use OpenMP to enable parallel execution of our codes. If multiple workers can do the same job, execution will be sped up.

In [ ]:
pygmentize omp_examples/a01-parallel.cpp

We need to tell the compiler that our program contains OpenMP pragmas with the `-fopenmp` option.

In [ ]:
g++ omp_examples/a01-parallel.cpp -fopenmp -o parallel

Now we can run the generated executable.

In [ ]:
./parallel

# Infos about the region
OpenMP allows to get the information about number of threads present in each parallel region

In [ ]:
pygmentize omp_examples/a02-infos.cpp

In [ ]:
g++ omp_examples/a02-infos.cpp -fopenmp -o infos && ./infos

# Setting the number of threads
We can do this either on each parallel region or with an environment variable

In [ ]:
pygmentize omp_examples/a03-threadnum.cpp

In [ ]:
g++ omp_examples/a03-threadnum.cpp -fopenmp -o threadnum && ./threadnum

We can set the number of threads to use when nothing is specified with the `OMP_NUM_THREADS` environment variable.

In [ ]:
OMP_NUM_THREADS=4 ./infos

In [ ]:
OMP_NUM_THREADS=4 ./threadnum

# Parallel loops
Since loops are such an important concept to parallelize, there is a special directive for it

In [ ]:
pygmentize omp_examples/a04-loops.cpp

In [ ]:
g++ omp_examples/a04-loops.cpp -fopenmp -o loops

In [ ]:
OMP_NUM_THREADS=4 ./loops

In [ ]:
OMP_NUM_THREADS=12 ./loops

# Scheduling
Controlling the loop execution with the schedule directive

In [ ]:
pygmentize omp_examples/a05-schedule.cpp

In [ ]:
g++ omp_examples/a05-schedule.cpp -fopenmp -o schedule

In [ ]:
OMP_NUM_THREADS=2 ./schedule

# Variable Scoping
Who owns wich variables and what is their value 

In [ ]:
pygmentize omp_examples/a06-scoping.cpp

In [ ]:
g++ omp_examples/a06-scoping.cpp -fopenmp -o scoping

In [ ]:
./scoping

The private directive allows each thread to have a copy of a variable

In [ ]:
pygmentize omp_examples/a07-private.cpp

In [ ]:
g++ omp_examples/a07-private.cpp -fopenmp -o private

In [ ]:
OMP_NUM_THREADS=10 ./private

shared is the default but can also be stated explicitly

In [ ]:
pygmentize omp_examples/a08-shared.cpp

In [ ]:
g++ omp_examples/a08-shared.cpp -fopenmp -o shared

In [ ]:
OMP_NUM_THREADS=10 ./private

# Special Regions
We need special regions

In [ ]:
pygmentize omp_examples/a09-regions.cpp

In [ ]:
g++ omp_examples/a09-regions.cpp -fopenmp -o regions && ./regions

# What if parallel regions span multiple tasks
Barrier can be a thing

In [ ]:
pygmentize omp_examples/a10-barrier.cpp

In [ ]:
g++ omp_examples/a10-barrier.cpp -fopenmp -o barrier && ./barrier

In [ ]:
pygmentize omp_examples/a11-barrier.cpp

In [ ]:
g++ omp_examples/a11-barrier.cpp -fopenmp -o barrier2 && ./barrier2

# Nowait
and the opposite of it

In [ ]:
pygmentize omp_examples/a12-loops.cpp

In [ ]:
g++ omp_examples/a12-loops.cpp -fopenmp -o loops && ./loops

In [ ]:
pygmentize omp_examples/a13-nowait.cpp

In [ ]:
g++ omp_examples/a13-nowait.cpp -fopenmp -o nowait && ./nowait

# Reductions
Since reductions are such an omnipresent motif, we do not want to implement it with critical / atomic every time, so there is a keyword for it:

In [ ]:
pygmentize omp_examples/a14-reduction.cpp

In [ ]:
g++ omp_examples/a14-reduction.cpp -fopenmp -o reduction && ./reduction

# Amdahl's law

We look at the performance of the simple code above (slightly changed for better output readability

### Example

Computing $\pi$ using the Leibniz formula:
$$1 - \frac{1}{3} + \frac{1}{5} - \frac{1}{7} + \frac{1}{9} ... = \frac{\pi}{4}$$

In [ ]:
pygmentize omp_examples/b01-timing.cpp

In [ ]:
g++ -fopenmp omp_examples/b01-timing.cpp -o timing

In [ ]:
./timing 1 > out.txt
./timing 2 >> out.txt
./timing 3 >> out.txt
./timing 4 >> out.txt
./timing 5 >> out.txt
./timing 6 >> out.txt
./timing 7 >> out.txt
./timing 8 >> out.txt
./timing 9 >> out.txt
./timing 10 >> out.txt
./timing 11 >> out.txt
./timing 12 >> out.txt

In [ ]:
gnuplot -e "\
set terminal png; \
set style fill solid; \
set yrange[0:0.1]; \
set xlabel '# cores'; \
set ylabel 'runtime [s]'; \
plot 'out.txt' using 2: xtic(1) title 'runtime' with histogram \
" | display

In [ ]:
base=`head -1 out.txt | awk '{print $2}'`
gnuplot -e "\
set terminal png; \
set style fill solid; \
set yrange[0:14]; \
set xlabel '# cores'; \
set ylabel 'speedup (relative to 1 core)'; \
plot 'out.txt' using ($base/\$2): xtic(1) title 'runtime' with histogram, 
'out.txt' using :(\$1) title 'linear' with lines\
" | display

## Caching

Here we see the implication of caching in a multithreaded environment

In [ ]:
pygmentize omp_examples/b02-caching.cpp

In [ ]:
g++ -fopenmp omp_examples/b02-caching.cpp -o caching

In [ ]:
./caching 1 > caching.txt
./caching 2 >> caching.txt
./caching 4 >> caching.txt
./caching 8 >> caching.txt
./caching 12 >> caching.txt
./caching 13 >> caching.txt
./caching 20 >> caching.txt
./caching 24 >> caching.txt

In [ ]:
gnuplot -e "\
set terminal png; \
set style fill solid; \
set yrange[0:0.1]; \
set xlabel '# cores'; \
set ylabel 'runtime [s]'; \
plot 'caching.txt' using 2: xtic(1) title 'runtime' with histogram \
" | display